In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../data/iris.csv")
df = df.drop("Id", axis=1)
df.head()

In [ ]:
fig = df[df.Species == "Iris-setosa"].plot(
    kind="scatter", x="SepalLengthCm", y="SepalWidthCm", color="orange", label="Setosa"
)
df[df.Species == "Iris-versicolor"].plot(
    kind="scatter",
    x="SepalLengthCm",
    y="SepalWidthCm",
    color="blue",
    label="versicolor",
    ax=fig,
)
df[df.Species == "Iris-virginica"].plot(
    kind="scatter",
    x="SepalLengthCm",
    y="SepalWidthCm",
    color="green",
    label="virginica",
    ax=fig,
)
fig.set_xlabel("Sepal Length")
fig.set_ylabel("Sepal Width")
fig.set_title("Sepal Length VS Width")
fig = plt.gcf()
fig.set_size_inches(12, 8)
plt.show()

In [ ]:
train, test = train_test_split(df, test_size=0.3)
features_columns = [
    "SepalLengthCm",
    "SepalWidthCm",
    "PetalLengthCm",
    "PetalWidthCm",
]
X_train = train[features_columns]
y_train = train.Species
X_test = test[features_columns]
y_test = test.Species

# KNN Parameters

# Cross-Validation

# KNN System Requirements

# Model Evaluation

# Model Use Case